# JAZZ

Implementation of an automatic jazz improvisor that generates music for a jazz trio of piano, acoustic bass and percussion.  The code is derived from a program written by Erik Flister as a project for his undergraduate computer music class at CCRMA, Stanford University. 

Running this notebook requires a jupyter kernel that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
import sys 
sys.path.append('/Users/taube/Software/musx')
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import types
from musx import version, setmidiplayer, playfile, Score, Seq, Note, MidiFile, \
                 keynum, Cycle, Choose, Shuffle, intempo, odds, pick, between
from musx.midi.gm import AcousticGrandPiano, AcousticBass
print(f'musx version:', version)

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")
print('OK!')

The scale used by the improvisor (Dorian mode with a decorated octave):

In [ ]:
jazz_scale = [0, 2, 3, 5, 7, 9, 10, 12, 14]
print(f"jazz_scale: {jazz_scale}")

The chord changes for the piano and bass parts:

In [ ]:
jazz_changes = keynum('bf3 ef4 bf3 bf ef4 ef bf3 bf f4 ef bf3 bf')
print(f"jazz_changes: {jazz_changes}")

The tempo of the composition:

In [ ]:
jazz_tempo = 120
print(f"jazz_tempo: {jazz_tempo}")

## The percussion parts

The percussion parts for the Jazz Combo consist of two ride cymbals, a high hat, snare and bass drums. We will introduce these parts in their order of complexity, from simplest to most difficult.

##### Jazz High Hat: #####

In [ ]:
def jazz_high_hat(score, tmpo, ampl):
    """
    Plays the High Hat on the second and fourth quarter of every measure and
    rests on the first and third beats. Each sound lasts for the duration one
    triplet eighth note i.e. 1/3 of a beat.
    """
    rhy = intempo(1, tmpo)
    dur = intempo(1/3, tmpo)
    amp = .5
    pat = Cycle(['r', 42, 'r', 42]) # 'r' is rest
    for _ in range(4):
        x = pat.next()
        if x != 'r':
            m = Note(time=score.now, duration=dur, pitch=x, amplitude=amp * ampl, instrument=9)
            score.add(m)
        yield rhy
print(f"jazz_high_hat: {jazz_high_hat}")

Preview eight measures of the `jazz_high_hat()`. Since the process generates only one measure, we collect eight "versions" of the process and offset each by two seconds, exactly the duration of the combo's 4/4 measure at tempo 120:

In [ ]:
s = Score(out=Seq())
s.compose( [[t, jazz_high_hat(s, 120, 1)] for t in range(0, 15, 2)] )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing jazz_high_hat()...")

#### Jazz Drum: ####

In [ ]:
def jazz_drums(score, tmpo, ampl):
    """
    Randomly selects between playing the snare, the bass drum or resting one
    quarter of the time. One tenth of the time it produces a very loud tone.
    """
    elec_snare = 40
    bass_drum = 35
    knums = Choose(['r', elec_snare, bass_drum], [.25, 1, 1])
    rhys = Cycle([2/3, 1/3])
    amps = Choose([.7, .95], [1, .1])
    for _ in range(8):
        k = knums.next()
        a = amps.next()
        r = intempo(rhys.next(), tmpo)
        if k != 'r':
            m = Note(time=score.now, duration=r, pitch=k, amplitude=a * ampl, instrument=9)
            score.add(m)
        yield r
print(f"jazz_drums: {jazz_drums}")

Preview eight measures of the drum and hi hat together:

In [ ]:
s = Score(out=Seq())
m =  [ [ [t, jazz_high_hat(s, 120, .99)], [t, jazz_drums(s, 120, .50)]] for t in range(0, 15, 2) ] 
c = []
for x in m: c.extend(x)
s.compose(c)
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing drum and high hat...")

#### Jazz Cymbals: ####

In [ ]:
def jazz_cymbals(score, tmpo, ampl):
    """
    The cymbals process performs a constant stream of triplet eighths in
    which the ride1 cymbal is played on the beginning of every quarter
    note. The second and third triplets of each beat are either rests or
    a random choice between ride1, ride2 or a rest.  This is the beat
    map for a measure of the process, where '1' means the ride cymbal 1 is
    played, '-' means a rest, and 'x' means a random choice between ride1,
    ride2 or a rest:

    ```text
    Triplet 8th: 1  2  3    4  5  6    7  8  9   10 11 12
    Cymbals:     1  -  x    1  -  1    1  x  x    1  x  1 
    ```
    """
    ride1 = 51
    ride2 = 59
    rhy = intempo(1/3, tmpo)
    amps = Cycle([.6, .5, .9, .7, .5, 1, .6, .5, .9, .7, .5, 1])

    def subpat(wt):
        r1 = Choose([ride1, 'r'], [1, wt])
        r2 = Choose([ride2, 'r'], [1, wt])
        return Choose([r1, r2], [1.5, 1])

    # the events that happen on each triplet of the measure
    meas = {0: ride1,  1: 'r',        2: subpat(5),
            3: ride1,  4: 'r',        5: ride1,
            6: ride1,  7: subpat(7),  8: subpat(7),
            9: ride1, 10: subpat(3), 11: ride1}
    for b in meas:
        k = meas[b]
        if k != 'r':
            if type(k) is not int: # k is a subpattern
                k = k.next() #next(next(k))
            if k != 'r':
                a = amps.next()
                m = Note(time=score.now, duration=rhy, pitch=k, amplitude=a*ampl, instrument=9)
                score.add(m)
        yield rhy
print(f"jazz_cymbals: {jazz_cymbals}")

Preview all three percussion parts together:

In [ ]:
s = Score(out=Seq())
m =  [ [ [t, jazz_high_hat(s, 120, .99)], [t, jazz_drums(s, 120, .50)], [t, jazz_cymbals(s, 120, .50)]] 
      for t in range(0, 15, 2) ] 
c = []
for x in m: c.extend(x)
s.compose(c)
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing drum, high hat, and cymbals...")

#### Jazz Piano: ####

In [ ]:
def jazz_piano(score, on, tmpo, ampl):
    """
    The jazz piano improvises jazz chords based on a pattern of root
    changes and a scale pattern that is transposed to each root. The
    piano randomly choose between playing triplet eighths or straight
    eights for a given measure.
    """
    reps = odds(.65, 8, 12)
    scal = Shuffle(jazz_scale)
    rhys = Cycle([2/3, 1/3] if reps == 8 else [1/3])
    for _ in range(reps):
        r = intempo(rhys.next(), tmpo)
        #  two fifths of the time the piano will rest otherwise it plays a chord.
        l = [] if odds(2/5) else [scal.next() for _ in range(between(1,9))]
        for k in l:
            a = pick(.4, .5, .6, .7, .8)
            m = Note(time=score.now, duration=r, pitch=on+k, amplitude=a, instrument=0)
            score.add(m)
        yield r
print(f"jazz_piano: {jazz_piano}")

Preview the piano:

In [ ]:
s = Score(out=Seq())
s.compose( [[t, jazz_piano(s, keynum("bf3"), 120, 1)] for t in range(0, 15, 2)] )
playfile( MidiFile("temp.mid", s.out).write().pathname )
print("Previewing jazz_piano()...")

#### Jazz Bass: ####

In [ ]:
def jazz_bass(score, on, tmpo, ampl):
    """
    The bass part plays a melodic line built out of tones from the jazz-scale's
    tonic seventh chord alternating with color tones outside the tonic chord.
    The bass plays a series of 12 triplets per measure, on each triplet only one of
    the two sets is possible. On all but the first triplet a rest is also possible.
    """
    # 5 possible patterns for triplets 1-4
    a = Choose(['trrc', 'trrr', 'trtc', 'tctc', 'tctr'], [1.0, .25, .22, .065, .014])
    # 5 possible patterns for 5-7
    b = Choose(['rrt', 'rrr', 'rct', 'tct', 'tcr'], [1.0, .25, .22, .038, .007])
    # 5 possible patterns for 8-10
    c = Choose(['rrc', 'rtc', 'rrr', 'ctc', 'ctr'], [1.0, .415, .25, .11, .018])
    # two possible values for 11
    d = Choose(['r', 't'], [1, .25])
    # two possible values for 12
    e = Choose(['r', 'c'], [1, .25])
    # the measure map
    meas = a.next() + b.next() + c.next() + d.next() + e.next()

    rhy = intempo(1/3, tmpo)
    tonics = Choose([jazz_scale[i] for i in [0, 2, 4, 6, 7]])
    colors = Choose([jazz_scale[i] for i in [1, 3, 5, 6, 8]])
    amps = Cycle([.5, .4, 1.0, .9, .4, .9, .5, .4, 1.0, .9, .5, .9])
    durs = Cycle([2/3, 1/3, 1/3])

    for x in meas:
        k = -1
        if x == 't':
            k = tonics.next()
        elif x == 'c':
            k = colors.next()
        if k > -1:
            a = amps.next()
            d = durs.next()
            m = Note(time=score.now, duration=d, pitch=on+k, amplitude=ampl*a, instrument=1)
            score.add(m)
        yield rhy
print(f"jazz_bass: {jazz_bass}")

Preview the Jazz Bass:

In [ ]:
s = Score(out=Seq())
b =  MidiFile.metatrack(ins={1: AcousticBass})
s.compose( [[t, jazz_bass(s, keynum("bf1"), 120, 1)] for t in range(0, 15, 2)] )
playfile( MidiFile("temp.mid", [b, s.out]).write().pathname )
print("Previewing jazz_base()...")

Define the jazz combo conductor process:

In [ ]:
def jazz_combo(score, measures, tempo):
    """
    The conductor process adds parts for each measure to the score, so that changes
    to the overall texture, amplitude etc, could be added as the pieces progresses.
    """ 
    roots = Cycle(jazz_changes)
    ampl = .9
    for meas in range(measures):
        root = roots.next()
        if  0 == meas % 12:
           ampl = between(.5, 1)
        score.compose(jazz_piano(score, root, tempo, ampl))
        score.compose(jazz_cymbals(score, tempo, ampl))
        score.compose(jazz_high_hat(score, tempo, ampl))
        score.compose(jazz_drums(score, tempo, ampl))
        # shift the bass down one or two octaves
        score.compose(jazz_bass(score, odds(.5, root-12, root-24), tempo, ampl))
        yield intempo(4, tempo)
print(f"jazz_combo: {jazz_bass}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: AcousticGrandPiano, 1: AcousticBass})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1: {track1}")

Create a score and pass it track1 to receive the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Create the composition:

In [ ]:
score.compose(jazz_combo(score, 48, 120))
print("OK!")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("jazz.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if a terminal based midi player is installed:

In [ ]:
playfile(file.pathname)